# Data Encoder
Uses an autoencoder to reduce the dimensionality of the data.

In [1]:
import tensorflow as tf
print(f"GPU is {'not ' if len(tf.config.list_physical_devices('GPU')) == 0 else ''}available")

2024-05-07 10:14:11.508268: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-07 10:14:12.041952: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


GPU is available


2024-05-07 10:14:12.785939: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-07 10:14:12.814643: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-07 10:14:12.814687: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
import keras

# Data Processing

The data should already be present as `dataset.csv` and `top_unigrams.txt` in the `data` folder.

If they are not present, do the following.
1. Ensure that the VirusTotal reports are present in `data/json` with the format `[LABEL]_[HASH].json`.
3. Run `prepare_data.py`. This will generate the two files needed for this notebook.

In [3]:
import pandas as pd

In [4]:
raw_df = pd.read_csv("../data/dataset.csv")
raw_df

,label,hash,dim-0000,dim-0001,dim-0002,dim-0003,dim-0004,dim-0005,dim-0006,dim-0007,...,dim-9990,dim-9991,dim-9992,dim-9993,dim-9994,dim-9995,dim-9996,dim-9997,dim-9998,dim-9999
0,TRICKBOT,f16631469eb35406ef4049d30c763cadda571b25bbdb45...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,DARKKOMET,d31a7102cbc54447c251ba62760eb484fd0c9fbb8ea54f...,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,SALITY,e7fc7de574f44a966b198b7625bd6c595cad05bd669619...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ADPOSHEL,fb576aea86528eaa082efbd073a7d4a6d1c2006da9ba49...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,VEBZENPAK,4519186b8fb2eaa847255087b44f918928c20e97c2fbea...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5185,TRICKBOT,3cb4905c742c52b55c8930003224d4176220950c4cc37b...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5186,BENIGN,958cceb0f7f7ae76b2527744da7e2305a372aff304d372...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5187,TRICKBOT,46401903e85a5c457490a6934ec4dc61fdf28df83af377...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5188,TRICKBOT,7eca38a5d0098a7ca4baa1faca43b80b5f911b7580273b...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


For the training of the model, we don't need the label or the file hash.

In [5]:
df = raw_df.drop(columns=["label", "hash"])
df

,dim-0000,dim-0001,dim-0002,dim-0003,dim-0004,dim-0005,dim-0006,dim-0007,dim-0008,dim-0009,...,dim-9990,dim-9991,dim-9992,dim-9993,dim-9994,dim-9995,dim-9996,dim-9997,dim-9998,dim-9999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5185,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5186,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5187,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5188,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


80% of the dataframe will be saved for training, while 20% will be left for testing.

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(
    df, test_size=0.2, random_state=42
)

In [7]:
X_train.shape

(4152, 10000)

In [8]:
X_test.shape

(1038, 10000)

# Model Creation

We will use an autoencoder to reduce the dimensionality of the data.

In [9]:
from keras import layers
from keras.models import Sequential

In [10]:
LAYER_SIZES = [512, 256, 128, 32]  # The last layer is the center layer

In [11]:
INITIAL_EPOCH = 0

def create_encoder():
    model = Sequential(name="Encoder")
    model.add(keras.Input((df.shape[1],), name="encoder-input"))

    for layer_size in LAYER_SIZES[:-1]:
        model.add(layers.Dense(layer_size, activation="relu"))

    # Add an activity regularizer to make the middle layer sparse
    model.add(layers.Dense(LAYER_SIZES[-1], activation="relu"))
    
    return model


def create_decoder():
    model = Sequential(name="Decoder")
    model.add(keras.Input((LAYER_SIZES[-1],), name="decoder-input"))

    for layer_size in LAYER_SIZES[-2::-1]:  # Starting from second last
        model.add(layers.Dense(layer_size, activation="relu"))
    model.add(layers.Dense(df.shape[1], activation="relu"))
    
    return model


def create_autoencoder(encoder, decoder):
    model = Sequential(name="Autoencoder")
    model.add(keras.Input((df.shape[1],), name="encoder-input"))
    model.add(encoder)
    model.add(decoder)

    model.compile(
        loss="mse",
        optimizer="adam",
        metrics=["mae"]
    )

    return model

In [12]:
encoder = create_encoder()
decoder = create_decoder()
autoencoder = create_autoencoder(encoder, decoder)

2024-05-07 10:14:17.474605: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-07 10:14:17.474691: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-07 10:14:17.474721: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-07 10:14:18.238330: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-07 10:14:18.238463: I external/local_xla/xla/stream_executor

In [13]:
autoencoder.summary(expand_nested=True)

Model: "Autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Encoder (Sequential)            │ (None, 32)             │     5,288,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense (Dense)              │ (None, 512)            │     5,120,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_1 (Dense)            │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_2 (Dense)            │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_3 (Dense)            │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Decoder (Sequential)            │ (None, 10000)          │     5,298,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_4 (Dense)            │ (None, 128)            │         4,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_5 (Dense)            │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_6 (Dense)            │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_7 (Dense)            │ (None, 10000)          │     5,130,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,587,696 (40.39 MB)

 Trainable params: 10,587,696 (40.39 MB)

 Non-trainable params: 0 (0.00 B)

Define callbacks.

In [14]:
import os

checkpoint_path = "../models/encoder/checkpoint.keras"
checkpoint_dir = os.path.dirname(checkpoint_path)

checkpointer = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

In [15]:
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, verbose=1, min_delta=1e-4)

Load latest checkpoint if there is one.

In [16]:
# INITIAL_EPOCH = 54
# autoencoder = keras.models.load_model(checkpoint_path)

Train the model.

In [17]:
NUM_EPOCHS = 200

autoencoder.fit(
    X_train,
    X_train,
    validation_split=0.2,
    initial_epoch=INITIAL_EPOCH,
    epochs=NUM_EPOCHS,
    callbacks=[checkpointer, early_stopping]
)

Epoch 1/200


I0000 00:00:1715048060.361127   71535 service.cc:145] XLA service 0x7f8ddc002c10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1715048060.361166   71535 service.cc:153]   StreamExecutor device (0): Quadro P1000, Compute Capability 6.1
2024-05-07 10:14:20.408749: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-07 10:14:21.637665: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8902


 16/104 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0195 - mae: 0.0272

I0000 00:00:1715048064.179891   71535 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0159 - mae: 0.0233
Epoch 1: val_loss improved from inf to 0.01080, saving model to ../models/encoder/checkpoint.keras
104/104 ━━━━━━━━━━━━━━━━━━━━ 10s 51ms/step - loss: 0.0159 - mae: 0.0233 - val_loss: 0.0108 - val_mae: 0.0154
Epoch 2/200
101/104 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0099 - mae: 0.0140
Epoch 2: val_loss improved from 0.01080 to 0.00917, saving model to ../models/encoder/checkpoint.keras
104/104 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0099 - mae: 0.0140 - val_loss: 0.0092 - val_mae: 0.0127
Epoch 3/200
101/104 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0089 - mae: 0.0124
Epoch 3: val_loss improved from 0.00917 to 0.00846, saving model to ../models/encoder/checkpoint.keras
104/104 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0089 - mae: 0.0124 - val_loss: 0.0085 - val_mae: 0.0116
Epoch 4/200
101/104 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0082 - mae: 0.0112
Epoch 4: val_loss improved from 0.00846 to 0.00811

Load the best performing model.

In [18]:
autoencoder = keras.models.load_model(checkpoint_path)

Evaluate the autoencoder on the test data.

In [19]:
test_mse, test_mae = autoencoder.evaluate(X_test, X_test, verbose=1)
print(f"Testing MSE: {test_mse:5.5f}")
print(f"Testing MAE: {test_mae:5.5f}")

33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.0058 - mae: 0.0077
Testing MSE: 0.00568
Testing MAE: 0.00752


Get only the encoder part to save.

In [20]:
encoder = autoencoder.get_layer("Encoder")

In [21]:
encoder.summary()

Model: "Encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 512)            │     5,120,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         4,128 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,288,864 (20.18 MB)

 Trainable params: 5,288,864 (20.18 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
encoder.save("../models/encoder/encoder.keras")

# Transforming Original Data

In [23]:
transformed_df = encoder.predict(df)

163/163 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [24]:
transformed_df = pd.DataFrame(transformed_df, columns=[f"dim-{i:02d}" for i in range(LAYER_SIZES[-1])])

Add the labels and hashes back to the dataframe.

In [25]:
transformed_df.insert(0, "label", raw_df["label"])
transformed_df.insert(1, "hash", raw_df["hash"])

In [26]:
transformed_df

,label,hash,dim-00,dim-01,dim-02,dim-03,dim-04,dim-05,dim-06,dim-07,...,dim-22,dim-23,dim-24,dim-25,dim-26,dim-27,dim-28,dim-29,dim-30,dim-31
0,TRICKBOT,f16631469eb35406ef4049d30c763cadda571b25bbdb45...,0.0,0.0,21.743380,0.0,22.543247,2.443337,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,6.863416,0.0,11.860749,0.000000,0.000000,6.950177
1,DARKKOMET,d31a7102cbc54447c251ba62760eb484fd0c9fbb8ea54f...,0.0,0.0,0.000000,0.0,5.224687,4.526221,0.000000,0.000000,...,0.000000,0.0,0.0,7.530015,13.983922,0.0,0.294426,0.000000,0.000000,16.371454
2,SALITY,e7fc7de574f44a966b198b7625bd6c595cad05bd669619...,0.0,0.0,8.111208,0.0,20.635378,5.108455,0.000000,4.035753,...,2.569614,0.0,0.0,0.000000,3.810240,0.0,15.515412,0.000000,1.036820,7.405690
3,ADPOSHEL,fb576aea86528eaa082efbd073a7d4a6d1c2006da9ba49...,0.0,0.0,15.510031,0.0,8.777089,0.000000,1.650506,0.000000,...,0.000000,0.0,0.0,0.000000,5.584435,0.0,9.167680,0.000000,0.000000,0.000000
4,VEBZENPAK,4519186b8fb2eaa847255087b44f918928c20e97c2fbea...,0.0,0.0,1.373834,0.0,0.853976,4.216197,2.825515,0.000000,...,0.000000,0.0,0.0,2.700450,2.623316,0.0,2.560539,0.000000,4.986362,15.735305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5185,TRICKBOT,3cb4905c742c52b55c8930003224d4176220950c4cc37b...,0.0,0.0,1.128719,0.0,4.236022,6.092369,0.000000,1.430992,...,3.938910,0.0,0.0,0.000000,3.109124,0.0,2.455679,0.000000,9.521128,6.103506
5186,BENIGN,958cceb0f7f7ae76b2527744da7e2305a372aff304d372...,0.0,0.0,5.199644,0.0,8.188759,0.000000,0.000000,4.260060,...,0.000000,0.0,0.0,0.000000,6.749435,0.0,5.007703,3.713518,12.038464,3.414638
5187,TRICKBOT,46401903e85a5c457490a6934ec4dc61fdf28df83af377...,0.0,0.0,16.154514,0.0,5.835102,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,12.208120,18.016140,0.0,4.018389,0.000000,0.000000,2.246171
5188,TRICKBOT,7eca38a5d0098a7ca4baa1faca43b80b5f911b7580273b...,0.0,0.0,7.892935,0.0,0.804787,10.592616,0.000000,0.000000,...,0.000000,0.0,0.0,1.198672,14.281110,0.0,0.000000,0.000000,0.000000,0.000000


In [27]:
transformed_df.to_csv("../data/encoded-data.csv", index=False)